# Basic ETL Workflow Example with CoinGecko Data

This notebook demonstrates a simple ETL (Extract, Transform, Load) process using the CoinGecko API:

- **Extract:** Download the top 5 cryptocurrencies by market cap.
    - In this phase, we set the URL and specify the parameters, which influence how our data is returned.
- **Transform:** Clean and select relevant columns for analysis.
- **Load:** Save the processed data to a CSV file for further use.

This workflow is a common pattern in data engineering for ingesting and preparing external data.

## Concepts Introduced
- **DataFrame**
    - A DataFrame is tabular data format commonly used in data processing with Python via the Pandas library.
- **Normalizing**
    - Normalizing in this context refers to converting complex or nested data (like JSON) into a flat table format (DataFrame) so it's easier to work with. More generally, it means to make data more consistent or easier to work with.

### Extract

In [1]:
# Extract
import requests

#  Get top 5 cryptocurrencies by market cap from CoinGecko
url = f"https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 5,
    "page": 1,
    "sparkline": False
}
response = requests.get(url, params=params)
data = response.json()

### Transform

In [2]:
# Transform
import pandas as pd

# Normalize and select relevant columns
if data:

    # Normalize JSON response into a DataFrame
    df = pd.json_normalize(data)

    # Select relevant columns
    df = df[["id", "symbol", "name", "current_price", "market_cap", "total_volume"]]
    
    # Remove rows with NaN values
    df = df.dropna()
else:
    df = pd.DataFrame()

# Convert DataFrame to a list of dictionaries
flattened_data = df.to_dict(orient="records")

# Display the first 5 rows of the DataFrame
df.head()

,id,symbol,name,current_price,market_cap,total_volume
0,bitcoin,btc,Bitcoin,107015.00,2127962538117,23800722336
1,ethereum,eth,Ethereum,2409.66,290679990723,12990101448
2,tether,usdt,Tether,1.00,157258177785,42373100460
3,ripple,xrp,XRP,2.12,124775929668,2823262199
4,binancecoin,bnb,BNB,644.07,93944495799,620601639


### Load

In [3]:
# Load
# For demonstration, save the transformed data to a CSV file
output_path = "top_5_cryptos.csv"
df.to_csv(output_path, index=False)
print(f"Saved top 5 cryptocurrencies data to {output_path}")

Saved top 5 cryptocurrencies data to top_5_cryptos.csv


### Load with SQLite

In [4]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('crypto_data.db')

# Save the DataFrame to a SQLite table
df.to_sql('top_cryptos', conn, if_exists='replace', index=False)
df_loaded = pd.read_sql('SELECT * FROM top_cryptos', conn)
conn.close()

# Display the DataFrame
display(df_loaded)
print('Saved top 5 cryptocurrencies data to the top_cryptos table in crypto_data.db')

,id,symbol,name,current_price,market_cap,total_volume
0,bitcoin,btc,Bitcoin,107015.00,2127962538117,23800722336
1,ethereum,eth,Ethereum,2409.66,290679990723,12990101448
2,tether,usdt,Tether,1.00,157258177785,42373100460
3,ripple,xrp,XRP,2.12,124775929668,2823262199
4,binancecoin,bnb,BNB,644.07,93944495799,620601639


Saved top 5 cryptocurrencies data to the top_cryptos table in crypto_data.db
